In [1]:
import pandas as pd 

data = pd.read_csv('train.csv')
data

,id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,0,McDonald's,Fast food restaurant,"9814 International Dr, Orlando, FL 32819, Unit...",28.423814,-81.461242,"5,468",11 months ago,"Very nice staff, good place to get all your fa...",5
1,1,McDonald's,Fast food restaurant,"1100 N US Hwy 377, Roanoke, TX 76262, United S...",33.009318,-97.222925,998,10 months ago,"I think the morning shift are one of, if not t...",5
2,2,McDonald's,Fast food restaurant,"490 8th Ave, New York, NY 10001, United States",40.752529,-73.992876,"3,902",a year ago,Horrible! Avoid this place at night. Unsafe. ...,1
3,3,McDonald's,Fast food restaurant,"111 Madison St, Oak Park, IL 60302, United States",41.879656,-87.777913,"1,454",9 months ago,Employees too busy window licking to get any o...,1
4,4,McDonald's,Fast food restaurant,"5725 W Irlo Bronson Memorial Hwy, Kissimmee, F...",28.333508,-81.513738,"5,566",4 years ago,I'm sorry to give only one star. But during ou...,1
...,...,...,...,...,...,...,...,...,...,...
26495,26495,McDonald's,Fast food restaurant,"1650 Washington Ave, Miami Beach, FL 33139, Un...",25.790295,-80.132077,"4,922",a year ago,Be careful. Rude sketchy employees.They will t...,1
26496,26496,McDonald's,Fast food restaurant,"1540 2nd St, Santa Monica, CA 90405, United St...",34.012219,-118.494540,"3,380",4 years ago,I expect crowding because it's close to the be...,2
26497,26497,McDonald's,Fast food restaurant,"210 5th S, Salt Lake City, UT 84106, United St...",40.759057,-111.897383,"3,243",6 months ago,Salt Lake City fast becoming a toilet.\nElect ...,1
26498,26498,McDonald's,Fast food restaurant,"429 7th Ave, New York, NY 10001, United States",40.750506,-73.990583,"2,052",3 years ago,Good food,4


In [2]:
Q1 = data['latitude '].quantile(0.25)
Q3 = data['latitude '].quantile(0.75)
IQR = Q3- Q1 

lower_bound = Q1 - 1.5*IQR 
upper_bound = Q3 + 1.5*IQR 

outliers = data.loc[(data['latitude ']<lower_bound)|(data['latitude ']>upper_bound)].shape[0]
print(outliers)

0


In [3]:
from sklearn.impute import SimpleImputer 

def imputing(dataset):
    float_cols = dataset.select_dtypes(include='float64').columns 
    simple_imputer = SimpleImputer(strategy='mean')
    dataset[float_cols] = simple_imputer.fit_transform(dataset[float_cols])
    return dataset

data = imputing(data)

In [4]:
def comma_remover(dataset):
    dataset['rating_count'] = dataset['rating_count'].str.replace(r'\D', '', regex=True).astype('int')
    return dataset

data = comma_remover(data)

In [5]:
import re 

def conversion_time(time):
    if 'a' in time.lower()[0]:
        if 'day' in time.lower():
            return 1 
        if 'month' in time.lower():
            return 30 
        else:
            return 365
    else:
        num = int(re.search(r'\d+', time).group())
        if 'day' in time.lower():
            return num 
        if 'month' in time.lower():
            return 30*num 
        else:
            return 365*num    
        
data['review_time'] = data['review_time'].apply(conversion_time)

In [6]:
display(data.head())
data.isnull().sum()

,id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,0,McDonald's,Fast food restaurant,"9814 International Dr, Orlando, FL 32819, Unit...",28.423814,-81.461242,5468,330,"Very nice staff, good place to get all your fa...",5
1,1,McDonald's,Fast food restaurant,"1100 N US Hwy 377, Roanoke, TX 76262, United S...",33.009318,-97.222925,998,300,"I think the morning shift are one of, if not t...",5
2,2,McDonald's,Fast food restaurant,"490 8th Ave, New York, NY 10001, United States",40.752529,-73.992876,3902,365,Horrible! Avoid this place at night. Unsafe. ...,1
3,3,McDonald's,Fast food restaurant,"111 Madison St, Oak Park, IL 60302, United States",41.879656,-87.777913,1454,270,Employees too busy window licking to get any o...,1
4,4,McDonald's,Fast food restaurant,"5725 W Irlo Bronson Memorial Hwy, Kissimmee, F...",28.333508,-81.513738,5566,1460,I'm sorry to give only one star. But during ou...,1


id               0
store_name       0
category         0
store_address    0
latitude         0
longitude        0
rating_count     0
review_time      0
review           0
rating           0
dtype: int64

In [7]:
num_cols = data.select_dtypes(include=['float64', 'int64']).columns
cat_cols = (data.drop(columns=['category', 'store_name'])).select_dtypes(include='object').columns 

def concatenate(dataset, cols):
    return dataset[cols].agg(' '.join, axis=1)

data_transformed = data.copy()
data_transformed['concatenated_words'] = concatenate(dataset=data, cols=cat_cols)

In [ ]:
X = data_transformed.drop(columns=['category', 'store_address', 'review', 'store_name', 'rating'])
y = data_transformed['rating']

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import RobustScaler 
from sklearn.compose import ColumnTransformer 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

column_transformer = ColumnTransformer([
        ('robust_scaler', RobustScaler(), X_train.select_dtypes(include=['float64', 'int64']).columns), 
        ('tfidf', TfidfVectorizer(stop_words='english', max_features=5000), 'concatenated_words')
    ])


X_train_transformed = column_transformer.fit_transform(X_train)
X_test_transformed = column_transformer.transform(X_test)

X_test_transformed.shape, X_train_transformed.shape

((5300, 5005), (21200, 5005))

In [16]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import classification_report

log_reg = LogisticRegression(max_iter=100000,class_weight='balanced', solver='saga',n_jobs=-1)

log_reg.fit(X_train_transformed, y_train)

y_pred = log_reg.predict(X_test_transformed)

In [17]:
print(classification_report(y_pred=y_pred, y_true=y_test))

              precision    recall  f1-score   support

           1       0.75      0.73      0.74      1498
           2       0.32      0.48      0.39       490
           3       0.51      0.49      0.50       764
           4       0.47      0.53      0.50       918
           5       0.78      0.64      0.70      1630

    accuracy                           0.61      5300
   macro avg       0.57      0.57      0.57      5300
weighted avg       0.64      0.61      0.62      5300



In [ ]:
from sklearn.svm import SVC 

svc = SVC()
svc.fit(X_train_transformed, y_train)

y_pred = svc.predict(X_test_transformed)
print(classification_report(y_pred=y_pred, y_true=y_test))

              precision    recall  f1-score   support

           1       0.59      0.93      0.72      1498
           2       0.84      0.11      0.19       490
           3       0.75      0.29      0.42       764
           4       0.62      0.31      0.41       918
           5       0.63      0.82      0.71      1630

    accuracy                           0.62      5300
   macro avg       0.69      0.49      0.49      5300
weighted avg       0.65      0.62      0.57      5300



In [12]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()
random_forest.fit(X_train_transformed,y_train)

y_pred = random_forest.predict(X_test_transformed)
print(classification_report(y_pred=y_pred, y_true=y_test))

              precision    recall  f1-score   support

           1       0.66      0.91      0.77      1498
           2       0.82      0.30      0.44       490
           3       0.73      0.42      0.53       764
           4       0.67      0.42      0.52       918
           5       0.66      0.83      0.74      1630

    accuracy                           0.67      5300
   macro avg       0.71      0.58      0.60      5300
weighted avg       0.69      0.67      0.65      5300



In [13]:
from sklearn.tree import DecisionTreeClassifier

decision = DecisionTreeClassifier()
decision.fit(X_train_transformed, y_train)

y_pred = random_forest.predict(X_test_transformed)
print(classification_report(y_pred=y_pred, y_true=y_test))

              precision    recall  f1-score   support

           1       0.66      0.91      0.77      1498
           2       0.82      0.30      0.44       490
           3       0.73      0.42      0.53       764
           4       0.67      0.42      0.52       918
           5       0.66      0.83      0.74      1630

    accuracy                           0.67      5300
   macro avg       0.71      0.58      0.60      5300
weighted avg       0.69      0.67      0.65      5300



In [37]:
data_test = pd.read_csv('test.csv')
data_test.head()

,id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review
0,0,McDonald's,Fast food restaurant,"8500 US-290, Austin, TX 78724, United States",30.329135,-97.660629,"1,406",2 years ago,The kids love be the Bigfries and The Check in...
1,1,McDonald's,Fast food restaurant,"1415 E State Rd, Fern Park, FL 32730, United S...",28.655350,-81.342692,"1,618",4 years ago,Excellent
2,2,McDonald's,Fast food restaurant,"490 8th Ave, New York, NY 10001, United States",40.752529,-73.992876,"3,902",6 years ago,No change on food standard from other Mac rest...
3,3,McDonald's,Fast food restaurant,"5725 W Irlo Bronson Memorial Hwy, Kissimmee, F...",28.333508,-81.513738,"5,567",a year ago,John was great and super friendly everyone els...
4,4,McDonald's,Fast food restaurant,"72-69 Kissena Blvd, Queens, NY 11367, United S...",40.727401,-73.812460,"2,193",6 years ago,Strange drive thru entrance; quick service. Pa...


In [38]:
data_test = imputing(data_test)
data_test = comma_remover(data_test)
data_test['review_time'] = data_test['review_time'].apply(conversion_time)

data_test_transformed = data_test.copy()
data_test_transformed['concatenated_words'] = concatenate(dataset=data_test, cols=cat_cols)
data_test_transformed.head()

,id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,concatenated_words
0,0,McDonald's,Fast food restaurant,"8500 US-290, Austin, TX 78724, United States",30.329135,-97.660629,1406,730,The kids love be the Bigfries and The Check in...,"8500 US-290, Austin, TX 78724, United States T..."
1,1,McDonald's,Fast food restaurant,"1415 E State Rd, Fern Park, FL 32730, United S...",28.655350,-81.342692,1618,1460,Excellent,"1415 E State Rd, Fern Park, FL 32730, United S..."
2,2,McDonald's,Fast food restaurant,"490 8th Ave, New York, NY 10001, United States",40.752529,-73.992876,3902,2190,No change on food standard from other Mac rest...,"490 8th Ave, New York, NY 10001, United States..."
3,3,McDonald's,Fast food restaurant,"5725 W Irlo Bronson Memorial Hwy, Kissimmee, F...",28.333508,-81.513738,5567,365,John was great and super friendly everyone els...,"5725 W Irlo Bronson Memorial Hwy, Kissimmee, F..."
4,4,McDonald's,Fast food restaurant,"72-69 Kissena Blvd, Queens, NY 11367, United S...",40.727401,-73.812460,2193,2190,Strange drive thru entrance; quick service. Pa...,"72-69 Kissena Blvd, Queens, NY 11367, United S..."
